In [6]:
import pandas as pd

#### S3에서 데이터 불러오기

In [7]:
code = "https://ais7.s3.ap-northeast-2.amazonaws.com/%EC%82%AC%EB%B3%B8(29)+-+%ED%99%9C%EC%9A%A9%EB%8D%B0%EC%9D%B4%ED%84%B0/codeBook_v3.xlsx"
codeBook = pd.read_excel(code)

In [8]:
kp20 = "https://ais7.s3.ap-northeast-2.amazonaws.com/%EC%82%AC%EB%B3%B8(29)+-+%ED%99%9C%EC%9A%A9%EB%8D%B0%EC%9D%B4%ED%84%B0/KP2020.csv"
kp2020 = pd.read_csv(kp20, encoding='cp949', index_col=False)

In [9]:
kp21 = "https://ais7.s3.ap-northeast-2.amazonaws.com/%EC%82%AC%EB%B3%B8(29)+-+%ED%99%9C%EC%9A%A9%EB%8D%B0%EC%9D%B4%ED%84%B0/KP2021.csv"
kp2021 = pd.read_csv(kp21, encoding='cp949', index_col=False)

In [10]:
npa = "https://ais7.s3.ap-northeast-2.amazonaws.com/%EC%82%AC%EB%B3%B8(29)+-+%ED%99%9C%EC%9A%A9%EB%8D%B0%EC%9D%B4%ED%84%B0/NPA2020.csv"
npa2020 = pd.read_csv(npa, encoding='cp949', index_col=False)

#### concat

In [11]:
display(codeBook.loc[(codeBook.코드명 == "보이스피싱")])

,No,컬럼명,컬럼 그룹,코드명,코드값
57,58,EVT_CL_CD,사건종별코드,보이스피싱,215


In [12]:
# 보이스피싱 코드값에 해당하는 값만 추출
voice20 = kp2020.loc[(kp2020.EVT_CL_CD == 215)]
voice21 = kp2021.loc[(kp2021.EVT_CL_CD == 215)]
voice_npa20 = npa2020.loc[(npa2020.EVT_CL_CD == 215)]

In [13]:
print(f"voice20: {voice20.shape}, voice21: {voice21.shape}, voice_npa20: {voice_npa20.shape}")

voice20: (1660, 10), voice21: (33829, 10), voice_npa20: (10779, 10)


In [14]:
# kp2020, kp2021의 보이스피싱 컬럼만 먼저 concat
concat_voice = pd.concat([voice20, voice21])
# concat_voice.sample(10)

In [15]:
# RECV_CPLT_DM 20xx년 으로 변환을 위해 문자열앞에 20을 붙여줌
concat_voice["RECV_CPLT_DM"] = ["20" + concat_voice["RECV_CPLT_DM"].iloc[x] for x in range(len(concat_voice["RECV_CPLT_DM"]))]

# RECV_CPLT_DM 을 datetime 형변환
concat_voice["RECV_CPLT_DM"] = pd.to_datetime(concat_voice["RECV_CPLT_DM"])

# RECV_DEPT_NM 컬럼이 NPA_CL 컬럼값과 같아서 제거
concat_voice = concat_voice.drop(["RECV_DEPT_NM"], axis=1)

concat_voice.sample(5)

,RECV_CPLT_DM,NPA_CL,EVT_STAT_CD,EVT_CL_CD,RPTER_SEX,HPPN_PNU_ADDR,HPPN_X,HPPN_Y,SME_EVT_YN
1833138,2022-06-28 13:34:54,19,10,215,2.0,NaN,NaN,NaN,NaN
2133098,2022-09-14 18:38:22,19,10,215,3.0,충청남도 아산시 음봉면 산동리(음봉면 ) 692-4,127.091555,36.847217,NaN
958456,2021-10-08 13:49:15,19,10,215,2.0,충청남도 아산시 둔포면 석곡리(행정:둔포면) 1575,127.063072,36.907315,Y
2567543,2023-01-09 19:56:41,19,10,215,1.0,서울특별시 강북구 수유동 (행정:수유3동 ) 172-245,127.028112,37.641420,NaN
2280399,2022-10-18 18:51:23,13,10,215,2.0,대전광역시 서구 갈마동 (행정:갈마1동 ) 316-42,127.367923,36.351000,NaN


In [16]:
# RECV_CPLT_DT, RECV_CPLT_TM 를 datetime 형변환을 위해 str로 형변환
# concat_voice와 콘캣시 타입에 맞추기위해 float로 형변환
voice_npa20 = voice_npa20.astype({"RECV_CPLT_DT" : "str", "RECV_CPLT_TM" : "str", "RPTER_SEX" : "float"})
voice_npa20.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10779 entries, 3221 to 1177944
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   RECV_CPLT_DT   10779 non-null  object 
 1   RECV_CPLT_TM   10779 non-null  object 
 2   NPA_CL         10779 non-null  int64  
 3   EVT_STAT_CD    10779 non-null  int64  
 4   EVT_CL_CD      10779 non-null  int64  
 5   RPTER_SEX      10779 non-null  float64
 6   HPPN_OLD_ADDR  10147 non-null  object 
 7   HPPN_X         10664 non-null  float64
 8   HPPN_Y         10664 non-null  float64
 9   SME_EVT_YN     6221 non-null   object 
dtypes: float64(3), int64(3), object(4)
memory usage: 926.3+ KB


In [17]:
# RECV_CPLT_TM 에 숫자가 6개가 아닐경우 0으로 채워줌
voice_npa20["RECV_CPLT_TM"] = [voice_npa20["RECV_CPLT_TM"].iloc[x].zfill(6) for x in range(len(voice_npa20["RECV_CPLT_TM"]))]

# RECV_CPLT_TM 시,분,초 단위로 나누기위해 슬라이싱을 통해 ':' 를 붙여줌
voice_npa20["RECV_CPLT_TM"] = [voice_npa20["RECV_CPLT_TM"].iloc[x][:2] + ":" + voice_npa20["RECV_CPLT_TM"].iloc[x][2:4] + ":" + voice_npa20["RECV_CPLT_TM"].iloc[x][4:] for x in range(len(voice_npa20["RECV_CPLT_TM"]))]

# 콘캣시키기 위해 RECV_CPLT_DM 컬럼을 생성하면서 RECV_CPLT_DT와 RECV_CPLT_TM를 합쳐줌
voice_npa20["RECV_CPLT_DM"] = voice_npa20["RECV_CPLT_DT"] + " " + voice_npa20["RECV_CPLT_TM"]

# RECV_CPLT_DM 을 datetime 형변환
voice_npa20["RECV_CPLT_DM"] = pd.to_datetime(voice_npa20["RECV_CPLT_DM"])

voice_npa20.sample(2)

,RECV_CPLT_DT,RECV_CPLT_TM,NPA_CL,EVT_STAT_CD,EVT_CL_CD,RPTER_SEX,HPPN_OLD_ADDR,HPPN_X,HPPN_Y,SME_EVT_YN,RECV_CPLT_DM
906694,20200911,12:30:39,19,10,215,1.0,충청남도 당진시 원당동(행정:당진3동) 1251,126.64295,36.90192,,2020-09-11 12:30:39
588980,20200619,09:11:37,13,10,215,2.0,대전광역시 유성구 반석동(행정:노은3동) 623,127.31162,36.39385,NaN,2020-06-19 09:11:37


In [18]:
# RECV_CPLT_DM을 생성하였으므로 RECV_CPLT_DT, RECV_CPLT_TM 는 drop
voice_npa20 = voice_npa20.drop(["RECV_CPLT_DT", "RECV_CPLT_TM"], axis=1)

# 콘캣시 컬럼이름값이 같아야 해서 컬럼이름 변경
voice_npa20 = voice_npa20.rename(columns={'HPPN_OLD_ADDR' : 'HPPN_PNU_ADDR'})

# concat_voice의 컬럼순서와 맞춤
voice_npa20 = voice_npa20[['RECV_CPLT_DM', 'NPA_CL', 'EVT_STAT_CD', 'EVT_CL_CD',
       'RPTER_SEX', 'HPPN_PNU_ADDR', 'HPPN_X', 'HPPN_Y', 'SME_EVT_YN']]

voice_npa20.sample(2)

,RECV_CPLT_DM,NPA_CL,EVT_STAT_CD,EVT_CL_CD,RPTER_SEX,HPPN_PNU_ADDR,HPPN_X,HPPN_Y,SME_EVT_YN
1133408,2020-11-09 11:08:43,19,10,215,2.0,충청남도 천안시 서북구 입장면 도림리(행정:입장면) 341-1,127.24501,36.91461,Y
545664,2020-06-11 18:58:52,19,10,215,3.0,충청남도 예산군 삽교읍 목리삽교읍 928,126.67540,36.66120,Y


In [19]:
# 머지이이
voice = pd.concat([concat_voice, voice_npa20])
# voice = merge_voice.merge(voice_npa20, how="left")
voice.sample(10)

,RECV_CPLT_DM,NPA_CL,EVT_STAT_CD,EVT_CL_CD,RPTER_SEX,HPPN_PNU_ADDR,HPPN_X,HPPN_Y,SME_EVT_YN
2333997,2022-11-02 10:23:22,19,10,215,2.0,NaN,NaN,NaN,NaN
330199,2021-04-22 16:54:11,13,10,215,1.0,대전광역시 서구 변동 (행정:변동),127.387961,36.326975,Y
268030,2021-03-30 15:43:01,19,10,215,2.0,충청남도 공주시 반죽동(중학동) 237-2,127.121366,36.452667,NaN
49970,2020-12-22 19:41:23,19,10,215,1.0,충청남도 천안시 서북구 성정동(행정:성정2동) 1319,127.141017,36.829574,NaN
1570930,2022-04-22 10:06:56,19,10,215,1.0,NaN,NaN,NaN,NaN
1485278,2022-03-25 09:22:02,31,10,215,3.0,세종특별자치시 세종시 한솔동 (한솔동 ) 950,127.260000,36.477778,NaN
2344845,2022-11-04 08:33:34,19,10,215,3.0,NaN,NaN,NaN,NaN
74101,2021-01-28 16:45:01,19,10,215,2.0,충청남도 보령시 죽정동(행정:대천1동) 781,126.604250,36.357270,NaN
353560,2021-04-27 17:43:58,13,10,215,2.0,대전광역시 유성구 원신흥동 (행정:원신흥동) 541-5,127.338379,36.337218,NaN
1568119,2022-04-22 17:25:31,19,10,215,1.0,NaN,NaN,NaN,NaN


#### NPA_CL -> 코드명

In [20]:
# 코드명으로 변경을 위해 str로 형변환
voice = voice.astype({"NPA_CL" : "str", "EVT_STAT_CD" : "str", "RPTER_SEX" : "str"})

In [21]:
voice.NPA_CL.unique()

array(['19', '13', '8', '31', '11', '15', '20', '23', '18', '12', '21',
       '9', '17', '10', '14', '16', '22'], dtype=object)

In [22]:
# NPA_CL 변경
voice["NPA_CL"] = voice["NPA_CL"].str.replace("19", "충남청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("13", "대전청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("31", "세종시")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("11", "인천청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("15", "경기남부청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("20", "전북청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("23", "경남청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("18", "충북청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("12", "광주청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("21", "전남청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("17", "강원청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("10", "대구청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("14", "울산청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("16", "경기북부청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("22", "경북청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("8", "서울청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("9", "부산청")

In [23]:
voice.NPA_CL.unique()

array(['충남청', '대전청', '서울청', '세종시', '인천청', '경기남부청', '전북청', '경남청', '충북청',
       '광주청', '전남청', '부산청', '강원청', '대구청', '울산청', '경기북부청', '경북청'],
      dtype=object)

#### EVT_STAT_CD -> 코드명

In [24]:
voice.EVT_STAT_CD.unique()

array(['10', '5', '7', '9'], dtype=object)

In [25]:
voice["EVT_STAT_CD"] = voice["EVT_STAT_CD"].str.replace("10", "종결요청")
voice["EVT_STAT_CD"] = voice["EVT_STAT_CD"].str.replace("5", "도착")
voice["EVT_STAT_CD"] = voice["EVT_STAT_CD"].str.replace("7", "종결요청")
voice["EVT_STAT_CD"] = voice["EVT_STAT_CD"].str.replace("9", "다청이첩")

#### RPTER_SEX -> 코드명

In [26]:
voice.RPTER_SEX.unique()

array(['1.0', '2.0', '3.0'], dtype=object)

In [27]:
voice["RPTER_SEX"] = voice["RPTER_SEX"].str.replace("1.0", "남성")
voice["RPTER_SEX"] = voice["RPTER_SEX"].str.replace("2.0", "여성")
voice["RPTER_SEX"] = voice["RPTER_SEX"].str.replace("3.0", "불상")

C:\Users\tpfl1\AppData\Local\Temp\ipykernel_20460\3265531194.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  voice["RPTER_SEX"] = voice["RPTER_SEX"].str.replace("1.0", "남성")
C:\Users\tpfl1\AppData\Local\Temp\ipykernel_20460\3265531194.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  voice["RPTER_SEX"] = voice["RPTER_SEX"].str.replace("2.0", "여성")
C:\Users\tpfl1\AppData\Local\Temp\ipykernel_20460\3265531194.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  voice["RPTER_SEX"] = voice["RPTER_SEX"].str.replace("3.0", "불상")


#### save

In [28]:
path = r"C:\Users\tpfl1\OneDrive\Desktop\SERi"
voice.to_csv(f"{path}/voice.csv", index=False, encoding='cp949')